In [1]:
import os 
import time
import numpy as np
import tensorflow as tf
from tensorflow.python.ops import summary_ops_v2
from tensorflow import keras
from tensorflow.keras import datasets,layers,models,optimizers,metrics

In [2]:
# 0:默认值，打印所有信息 1：屏蔽INFO信息 2：屏蔽INFO信息和WARNING信息 3：屏蔽INFO,WARNING,ERROR信息
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [3]:
def mnist_datasets():
    (x_train,y_train),(x_test,y_test) = datasets.mnist.load_data()
    #numpy默认是float64,tf默认是float32
    x_train,x_test = x_train/np.float32(255),x_test/np.float32(255)
    y_train,y_test = y_train.astype(np.int64),y_test.astype(np.int64)
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train))
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test,y_test))
    return train_dataset,test_dataset